<img src="https://dsiag.ch/images/dsi_rgb.png" alt="dsi logo" width="100" style="position: absolute; right: 0px;"/>

# Electricity production plants - Geo Visualisation


Data origins from https://opendata.swiss/en/dataset/elektrizitatsproduktionsanlagen


## Loading data 

In [1]:
import pandas as pd
from ipyleaflet import Map, Marker

epp = pd.read_csv('../data/ch.bfe.elektrizitaetsproduktionsanlagen/ElectricityProductionPlant.csv', parse_dates=['BeginningOfOperation']).set_index('xtf_id')


## Coordinate System Transformation

The x,y values in the dataset are based on the LV95 coordinate system. To render the data we first transform the data into WGS84 (latitute/longitude) coordinates

In [2]:
# Convert CH y/x to WGS lat
def CHtoWGSlat(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 2600000) / 1000000
    x_aux = (x - 1200000) / 1000000
    lat = (16.9023892 + (3.238272 * x_aux)) + \
            - (0.270978 * pow(y_aux, 2)) + \
            - (0.002528 * pow(x_aux, 2)) + \
            - (0.0447 * pow(y_aux, 2) * x_aux) + \
            - (0.0140 * pow(x_aux, 3)) 
    lat = (lat * 100) / 36
    return lat

# Convert CH y/x to WGS long
def CHtoWGSlng(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 2600000) / 1000000
    x_aux = (x - 1200000) / 1000000
    lng = (2.6779094 + (4.728982 * y_aux) + \
            + (0.791484 * y_aux * x_aux) + \
            + (0.1306 * y_aux * pow(x_aux, 2))) + \
            - (0.0436 * pow(y_aux, 3))
    lng = (lng * 100) / 36
    return lng

epp['lat'] = epp.apply(lambda row: CHtoWGSlat(row._x, row._y), axis=1)
epp['lng'] = epp.apply(lambda row: CHtoWGSlng(row._x, row._y), axis=1)
display(epp.head())


,Address,PostCode,Municipality,Canton,BeginningOfOperation,InitialPower,TotalPower,MainCategory,SubCategory,PlantCategory,_x,_y,lat,lng
xtf_id,,,,,,,,,,,,,,
5646,Rue des Creusets 41,1948,Fionnay,VS,1958-03-07,1872000.0,1872000.0,maincat_1,subcat_1,plantcat_6,2589880.0,1097661.0,46.030414,7.307939
5686,Binenweg 5,3904,Naters,VS,1969-09-01,349576.0,349576.0,maincat_1,subcat_1,plantcat_7,2644115.0,1131390.0,46.332452,8.011545
5726,Robbia 504G,7741,San Carlo,GR,1910-11-03,29150.0,29150.0,maincat_1,subcat_1,plantcat_2,2801863.0,1136379.0,46.348429,10.061387
5727,Via Principale 16,7744,Campocologno,GR,1907-03-01,55000.0,55000.0,maincat_1,subcat_1,plantcat_7,2808646.0,1123676.0,46.232152,10.143753
5730,Büdemli 65B,7240,Küblis,GR,1922-01-01,44200.0,44200.0,maincat_1,subcat_1,plantcat_7,2778481.0,1198505.0,46.913661,9.781899


# Sample IPLeaflet Map

Sample from https://ipyleaflet.readthedocs.io/en/latest/

In [3]:
center = (46.030414,7.307939)

m = Map(center=center, zoom=15)

marker = Marker(location=center, draggable=True)

m.add_layer(marker);

display(m)


Map(center=[46.030414, 7.307939], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

***
 ## Put Power Plants on Map 

<div class="alert alert-block alert-success">
<b>Exercise: Create a map with all nuclear power plants in Switzerland</b> 

1. Create a 'Marker` for each nuclear power plant

2. Put the total power into the `title` of the `Marker`  
    
3. (Optional) Display the 100 largest water power plants with different colors (depending on the total power) as `Circle`. 
    
 

    
    
</div>

*Hints*
- Use `pd.apply()` with `axis=1` to execute a function on each row of a `DataFrame`
- IPLeaflet Doc: https://ipyleaflet.readthedocs.io/en/latest/

***